In [ ]:
import snowflake.snowpark as snowpark
from snowflake.snowpark import Session
from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *
import pandas as pd
from datetime import datetime, timedelta

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# # Set warehouse if needed
# session.sql("USE WAREHOUSE YOUR_WAREHOUSE_NAME").collect()
# session.sql("USE DATABASE YOUR_DATABASE_NAME").collect() 
# session.sql("USE SCHEMA YOUR_SCHEMA_NAME").collect()


In [ ]:
def get_last_partition(session, table_name):
    """
    Get the last partition from a table in Snowflake
    Note: Snowflake doesn't use Hive-style partitions, so this might need adjustment
    based on your table structure
    """
    try:
        # If your table has a partition column, adjust this query accordingly
        result = session.sql(f"""
            SELECT MAX(partition_column) as last_partition 
            FROM {table_name}
        """).collect()
        
        if result:
            return result[0]['LAST_PARTITION']
        else:
            return None
    except Exception as e:
        print(f"Error getting last partition: {e}")
        return None
def get_last_partition(session, table_name):
    """
    Get the last partition from a table in Snowflake
    Note: Snowflake doesn't use Hive-style partitions, so this might need adjustment
    based on your table structure
    """
    try:
        # If your table has a partition column, adjust this query accordingly
        result = session.sql(f"""
            SELECT MAX(partition_column) as last_partition 
            FROM {table_name}
        """).collect()
        
        if result:
            return result[0]['LAST_PARTITION']
        else:
            return None
    except Exception as e:
        print(f"Error getting last partition: {e}")
        return None

def process_data(session, env):
    """
    Process breakage data in Snowflake equivalent to the original PySpark job
    Based on merge_revenue_ifrs_dd_breakage configuration
    """
    
    # Define tables - Snowflake uses database.schema.table format
    table_1 = f'{env["table_1"]["database"]}.{env["table_1"]["schema"]}.{env["table_1"]["table"]}'
    table_2 = f'{env["table_2"]["database"]}.{env["table_2"]["schema"]}.{env["table_2"]["table"]}'
    table_3 = f'{env["table_3"]["database"]}.{env["table_3"]["schema"]}.{env["table_3"]["table"]}'
    table_4 = f'{env["table_4"]["database"]}.{env["table_4"]["schema"]}.{env["table_4"]["table"]}'
    
    # Define periode (matching the catalog filter patterns)
    event_date = env["table_1"]["filter_d2"]  # day=2 (2 days ago)
    load_date = env["table_1"]["filter_d0"]   # day=0 (today)
    month_date = env["table_2"]["filter_month"]  # month=0 (current month)
    
    print(f"Running for event_date={event_date} and load_date={load_date}")
    
    # Snowflake SQL query - converted from the original PySpark SQL
    sql_query = f"""
    SELECT 
        trx_date,
        trx_date AS purchase_date,
        transaction_id,
        '' AS subs_id,
        a.msisdn,
        c1.price_plan_id::INTEGER AS price_plan_id,
        brand,
        1 AS pre_post_flag,
        cust_type_desc,
        cust_subtype_desc,
        customer_sub_segment,
        '' AS lac,
        '' AS ci,
        lacci_id,
        '' AS node,
        area_sales,
        a.region_sales,
        branch,
        subbranch,
        cluster_sales,
        provinsi,
        kabupaten,
        kecamatan,
        kelurahan,
        NULL AS lacci_closing_flag,
        business_id AS sigma_business_id,
        rules_id AS sigma_rules_id,
        sku,
        '' AS l1_payu,
        '' AS l2_service_type,
        a.allowance_sub_type AS l3_allowance_type,
        '' AS l4_product_category,
        '' AS l5_product,
        '' AS l1_ias,
        '' AS l2_ias,
        '' AS l3_ias,
        '' AS commercial_name,
        '' AS channel,
        '' AS pack_validity,
        SUM(breakage * allocation_rate)::DECIMAL(38,15) AS rev_per_usage,
        SUM(0)::DECIMAL(38,15) AS rev_seized,
        SUM(0)::INTEGER AS dur,
        SUM(0)::INTEGER AS trx,
        SUM(0)::BIGINT AS vol,
        NULL AS cust_id,
        profile_name,
        quota_name,
        '' AS service_filter,
        '' AS price_plan_name,
        '' AS channel_id,
        '' AS site_id,
        '' AS site_name,
        region_hlr,
        '' AS city_hlr,
        '{load_date}' AS load_date,
        a.event_date,
        SOURCE
    FROM (
        SELECT 
            trx_date,
            transaction_id,
            msisdn,
            brand,
            cust_type_desc,
            cust_subtype_desc,
            customer_sub_segment,
            lacci_id,
            area_sales,
            region_sales,
            branch,
            subbranch,
            cluster_sales,
            provinsi,
            kabupaten,
            kecamatan,
            kelurahan,
            business_id,
            rules_id,
            sku,
            allowance_sub_type,
            allowance,
            profile_name,
            quota_name,
            region_hlr,
            event_date,
            SUM(breakage) AS breakage,
            SUM(rev_seized) AS rev_seized,
            SOURCE
        FROM (
            SELECT 
                trx_date,
                transaction_id,
                msisdn,
                brand,
                cust_type_desc,
                cust_subtype_desc,
                customer_sub_segment,
                lacci_id,
                area_sales,
                region_sales,
                branch,
                subbranch,
                cluster_sales,
                provinsi,
                kabupaten,
                kecamatan,
                kelurahan,
                business_id,
                rules_id,
                sku,
                allowance_sub_type,
                allowance,
                profile_name,
                quota_name,
                region_hlr,
                event_date,
                SUM(breakage) AS breakage,
                '0' AS rev_seized,
                'BREAKAGE' AS SOURCE
            FROM {table_1} a
            WHERE event_date = '{event_date}'
            GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,28
            
            UNION ALL
            
            SELECT 
                trx_date,
                transaction_id,
                msisdn,
                brand,
                cust_type_desc,
                cust_subtype_desc,
                customer_sub_segment,
                lacci_id,
                area_sales,
                region_sales,
                branch,
                subbranch,
                cluster_sales,
                provinsi,
                kabupaten,
                kecamatan,
                kelurahan,
                sigma_business_id AS business_id,
                sigma_rules_id AS rules_id,
                sku,
                l3_allowance_type AS allowance_sub_type,
                CASE 
                    WHEN UPPER(l3_allowance_type) LIKE '%DATA%'
                      OR UPPER(l3_allowance_type) LIKE '%UPCC%' THEN 'DATA'
                    WHEN UPPER(l3_allowance_type) LIKE '%SMS%' THEN 'SMS'
                    WHEN UPPER(l3_allowance_type) LIKE '%VOICE%' THEN 'VOICE'
                    ELSE 'NONUSAGE' 
                END AS allowance,
                profile_name,
                quota_name,
                region_hlr,
                event_date,
                SUM(rev_seized * -1) AS breakage,
                '0' AS rev_seized,
                'BREAKAGE-SEIZED' AS SOURCE
            FROM {table_2}
            WHERE event_date = '{event_date}'
              AND SUBSTR(purchase_date, 1, 7) = '{month_date}'
              AND SOURCE IN ('SEIZED','UPCC-SEIZED')
              AND pre_post_flag = '1'
            GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,28
        ) a
        GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,29
    ) a
    
    LEFT JOIN (
        SELECT 
            event_date,
            msisdn,
            offer_id AS price_plan_id
        FROM {table_3}
        WHERE event_date = '{event_date}'
          AND pre_post_flag = '1'
        GROUP BY 1,2,3
    ) c1
    ON a.event_date = c1.event_date
       AND a.msisdn = c1.msisdn
    
    INNER JOIN (
        SELECT * 
        FROM {table_4}
        WHERE event_date IN (SELECT MAX(event_date) FROM {table_4})
    ) b1
    ON a.allowance = b1.allowance_type
       AND a.region_sales = b1.region
    
    GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,45,46,47,48,49,50,51,52,53,54,55,56,57
    """
    
    # Execute the query
    df = session.sql(sql_query)
    
    return df

In [ ]:
try:
    env = {
        "table_1": {
            "database": "TELKOMSEL_POC",
            "schema": "public",
            "table": "STG_BREAKAGE_POC_TOKENIZED",
            "filter_d2": "2025-04-01",     # Hardcoded: 1st April 2025 (event_date)
            "filter_d0": "2025-04-01"      # Hardcoded: 2nd April 2025 (load_date)
        },
        "table_2": {
            "database": "TELKOMSEL_POC",
            "schema": "RAW",
            "table": "MERGE_REVENUE_IFRS_DD_POC_TOKENIZED",
            "filter_d2": "2025-04-01",     # Hardcoded: 1st April 2025 (event_date)
            "filter_month": "2025-04"      # Hardcoded: April 2025 (month filter)
        },
        "table_3": {
            "database": "TELKOMSEL_POC",
            "schema": "RAW",
            "table": "MERGE_REVENUE_DD_POC_TOKENIZED",
            "filter_d2": "2025-04-01",     # Hardcoded: 1st April 2025 (event_date)
        },
        "table_4": {
            "database": "TELKOMSEL_POC",
            "schema": "RAW",
            "table": "BREAKAGE_REFERENCE_POC_TOKENIZED"
        }
    }
    
    # Process the data
    result_df = process_data(session, env)
    
    # Show results
    print("Processing completed successfully!")
    result_df.show(10)  # Show first 10 rows
    
except Exception as e:
    print(f"Error processing data: {e}")
    raise